In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly import colors as pcol
import plotly

write_args = dict(include_plotlyjs='/static/js/plotly.min.js', full_html=False)

In [ ]:
df = pd.read_csv('../results/log_bleak.csv')
df

In [ ]:
ble = df[df.dir.str.startswith('BLE')].reset_index()
uart = df[df.dir.str.startswith('UART')].reset_index()

In [ ]:
def base_fig(title:str):
    colors = pcol.sequential.YlGnBu[::2] if title.startswith('BLE') else pcol.sequential.OrRd[::2]
    _fig = go.Figure(layout=dict(
        title=title.replace('>>', '➟'), template='plotly_dark', 
        # width=900,
        margin=dict(l=100, r=100),
        hovermode='x',
        # legend={'x': 1, 'title': 'delay'},
        colorway=colors
    ))
    _fig.add_annotation(text='[CC BY] blog.ja-ke.tech', showarrow=False, font={'color': 'gray'},
        xref='paper', yref='paper', x=1.1, y=-.25)
    return _fig


In [ ]:
def create_bw(data: pd.DataFrame, title: str, packet: int):
    fig = base_fig(f'{title}: Throughput')
    for s in data.loss_percent:
        fig.add_scatter(x=data.index, y=data.rx_baud[s], 
            name=f'{int(10/1000/s*packet) if s > 0 else "∞"} kb/s')
    fig.update_xaxes(type='category', title='module baud rate')
    fig.update_yaxes(ticksuffix='b/s', title='bandwidth', rangemode='tozero')
    fig.update_layout(legend={'title': 'max. (from delay)'})
    return fig

def create_loss(data: pd.DataFrame, title: str, packet: int):
    fig = base_fig(f'{title}: Data Transmission')
    for s in data.loss_percent:
        fig.add_bar(x=data.index, y=data.loss_percent[s], 
             name=f'{s/packet/0.001:.1f} ms')
    fig.update_xaxes(type='category', title='module baud rate')
    # fig.update_yaxes(ticksuffix='%', title='packet loss', range=[0,100])
    fig.update_yaxes(ticksuffix='%', title='data received', rangemode='tozero')
    fig.update_layout(legend={'title': 'per byte delay'})
    return fig

In [ ]:
def gen_plots(inp: pd.DataFrame):
    title: str = inp.dir[0]
    p = inp.pivot(index='rated_baud', columns='delay', values=['loss_percent', 'rx_baud'])
    p.loss_percent = 100-p.loss_percent
    packet = inp.packet_size[0]

    loss = create_loss(p, title, packet)
    bw = create_bw(p, title, packet)

    write_name = title.replace(' >> ', '-').lower()
    loss.write_html(f'{write_name}_loss.html', **write_args)
    bw.write_html(f'{write_name}_bw.html', **write_args)

    return [loss, bw]

figs = gen_plots(ble) + gen_plots(uart)

In [ ]:
for f in figs:
    f.show()

In [ ]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# for s in p.loss_percent:
#     fig.add_scatter(x=p.index, y=p.rx_baud[s], marker_color=pcol.sequential.Viridis, name=f'{10/s*packet if s > 0 else "∞"} b/s max', secondary_y=True)
#     fig.add_bar(x=p.index, y=p.loss_percent[s], name=f'{s//0.001} ms delay/byte ')

# fig.update_layout(title='BLE ➟ UART: max. bandwidth', template='plotly_dark', 
#     width=900, legend={'x': 1.1})
# fig.update_xaxes(type='category', title='baud rate')
# fig.update_yaxes(ticksuffix='%', title='packet loss', secondary_y=False)
# fig.update_yaxes(ticksuffix='b/s', title='bandwidth', range=[0, 25e3], secondary_y=True)
# fig.add_annotation(text='[CC BY] blog.ja-ke.tech', showarrow=False,
#     xref='paper', yref='paper', x=1.28, y=-.25)
# fig.show()

In [ ]:
df_prev = pd.read_csv('../results/log_bluepy.csv')
df_prev

In [ ]:
def prepare_comp(data):
    data.loss_percent = 100-data.loss_percent
    data = data[data.delay == 0].reset_index()
    return data.pivot(index='rated_baud', columns='dir', values='loss_percent')

def add_comp(inp: pd.DataFrame, i, ver):
    ser = inp.iloc[:, i]
    co_fig.add_scatter(x=inp.index, y=ser, name=f"{ver}: {ser.name.replace('>>', '➟')}")


In [ ]:
old = prepare_comp(df_prev.copy())
new = prepare_comp(df.copy())

In [ ]:
co_fig = base_fig('BLE Serial 1.3 vs 2.0 [delay = 0 ms]')
co_fig.update_layout(colorway=["rgb(65,182,196)", "rgb(252,141,89)", "rgb(34,94,168)", "rgb(215,48,31)"])
add_comp(old, 0, '1.3'); add_comp(old, 1, '1.3')
add_comp(new, 0, '2.0'); add_comp(new, 1, '2.0')
co_fig.update_xaxes(type='category', title='module baud rate')
co_fig.update_yaxes(ticksuffix='%', title='data received', rangemode='tozero')
co_fig.show()

In [ ]:
co_fig.write_html('compare-2.html', **write_args)